In [1]:
import pandas as pd 
import openmatrix as omx
import numpy as np

import kpi

In [3]:
kpi.get_scenario_results('tnc_price', 'ex_6', 'scenario_1')

09/23/2022 05:13:34 PM root - INFO - Saving policy scenario resutls


{'policy': 'tnc_price',
 'name': 'scenario_1',
 'total_vmt': 21340.914593614812,
 'vmt_per_capita': 15.902320859623556,
 'vmt_per_capita_income': {'low': 14.515832063406275,
  'middle': 17.68010006317248,
  'high': 16.45092071505014},
 'vmt_per_capita_race': {'asian': 15.058961650634469,
  'black': 13.452365432145461,
  'other': 15.302133148879681,
  'white': 16.539728566122744},
 'vmt_per_capita_hispanic': {'hispanic': 14.594028279492939,
  'non_hispanic': 16.33409630975586},
 'vmt_per_capita_county': {'Alameda': 17.118590412909786,
  'Contra Costa': 18.795241876880695,
  'Marin': 18.19486510135572,
  'Napa': 26.041923099093967,
  'San Francico': 5.945400088929361,
  'San Mateo': 14.708025109017811,
  'Santa Clara': 15.360741426894332,
  'Solano': 22.968937777537615,
  'Sonoma': 16.02924617770694},
 'total_cs': 565828.6758924447,
 'cs_per_capita': 421.6309060301376,
 'cs_per_capita_income': {'low': 406.5314064176837,
  'middle': 490.9966325922382,
  'high': 364.54191251573974},
 'cs_p

In [ ]:
# scenario = '14_tnc_price_-025'

# def s3_path(scenario,table):
#     return 's3://carb-results/{}/final_{}.csv'.format(scenario,table)

# households = pd.read_csv(s3_path(scenario,'households'), index_col = 'household_id')
# persons = pd.read_csv(s3_path(scenario,'persons'), index_col = 'person_id')
# trips = pd.read_csv(s3_path(scenario,'trips'), index_col = 'trip_id')

In [ ]:
# # Working with a sample 
# scenario = 'ex_6'
# households = pd.read_csv('tmp/ex_6/final_households.csv', index_col = 'household_id')
# persons = pd.read_csv('tmp/ex_6/final_persons.csv', index_col = 'person_id')
# trips = pd.read_csv('tmp/ex_6/final_trips.csv', index_col = 'trip_id')

# scenario = 'ex_6'

# h_sample = households.sample(500)
# p_sample = persons[persons.household_id.isin(h_sample.index)]
# t_sample = trips[trips.household_id.isin(h_sample.index)]

# h_sample.to_csv('tmp/{}/households.csv'.format(scenario))
# p_sample.to_csv('tmp/{}/persons.csv'.format(scenario))
# t_sample.to_csv('tmp/{}/trips.csv'.format(scenario))
# skims = omx.open_file('tmp/{}/skims.omx'.format('ex_1'), 'r')

In [ ]:
scenario = 'ex_6'
households = pd.read_csv('tmp/ex_6/households.csv', index_col = 'household_id')
persons = pd.read_csv('tmp/ex_6/persons.csv', index_col = 'person_id')
trips = pd.read_csv('tmp/ex_6/trips.csv', index_col = 'trip_id', 
                    dtype = {'origin':int, 'destination':int})
skims = omx.open_file('tmp/ex_6/skims.omx'.format('ex_1'), 'r')
mapping = kpi.read_yaml('mapping.yaml')

In [ ]:
df, p_df = kpi.add_results_variables(mapping, trips, households, persons, skims)

In [ ]:
df.cs.plot.hist(bins = 100)

In [ ]:
persons

## VMT per capita

In [ ]:
def vmt_per_capita(trips, persons):
    n = len(persons)
    vmt_total = trips['VMT'].sum()
    return vmt_total/n

def vmt_per_capita_income(trips, persons):
    persons_segment = persons.groupby('income_category')['PNUM'].count()
    total_segment = trips.groupby('income_category').agg({'VMT':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['VMT']

def vmt_per_capita_race(trips, persons):
    persons_segment = persons.groupby('race')['PNUM'].count()
    total_segment = trips.groupby('race').agg({'VMT':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['VMT']

def vmt_per_capita_hispanic(trips, persons):
    persons_segment = persons.groupby('hispanic')['PNUM'].count()
    total_segment = trips.groupby('hispanic').agg({'VMT':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['VMT']

def vmt_per_capita_county(trips, persons):
    persons_segment = persons.groupby('lcm_county_id')['PNUM'].count()
    total_segment = trips.groupby('lcm_county_id').agg({'VMT':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['VMT']



In [ ]:
vmt_per_capita(df, p_df)

In [ ]:
vmt_per_capita_income(df, p_df)

In [ ]:
vmt_per_capita_race(df, p_df)

In [ ]:
vmt_per_capita_hispanic(df, p_df)

In [ ]:
vmt_per_capita_county(df, p_df)

## Consumer Surplus 

In [ ]:
def total_consumer_surplus(trips):
    return float(trips['cs'].sum())

def consumer_surplus_per_capita(trips, persons):
    n = len(persons)
    total = trips['cs'].sum()
    return float(total/n)

def consumer_surplus_per_capita_income(trips, persons):
    persons_segment = persons.groupby('income_category')['PNUM'].count()
    total_segment = trips.groupby('income_category').agg({'cs':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['cs']

def consumer_surplus_per_capita_race(trips, persons):
    persons_segment = persons.groupby('race')['PNUM'].count()
    total_segment = trips.groupby('race').agg({'cs':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['cs']

def consumer_surplus_per_capita_hispanic(trips, persons):
    persons_segment = persons.groupby('hispanic')['PNUM'].count()
    total_segment = trips.groupby('hispanic').agg({'cs':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['cs']

def consumer_surplus_per_capita_county(trips, persons):
    persons_segment = persons.groupby('lcm_county_id')['PNUM'].count()
    total_segment = trips.groupby('lcm_county_id').agg({'cs':'sum'})
    kpi = total_segment.div(persons_segment, axis = 0)
    return kpi.to_dict()['cs']

In [ ]:
consumer_surplus_per_capita(df, p_df)

In [ ]:
consumer_surplus_per_capita_income(df, p_df)

In [ ]:
consumer_surplus_per_capita_race(df, p_df)

In [ ]:
consumer_surplus_per_capita_hispanic(df, p_df)

In [ ]:
consumer_surplus_per_capita_county(df, p_df)

## Others

In [ ]:
def transit_ridersip(trips):
    return int(trips.carb_mode.isin(['Public Transit']).sum())

def mode_shares(trips):
    ms = trips.carb_mode.value_counts(normalize=True)
    return ms.to_dict()

def average_vehicle_ownership(households):
    return float(households.auto_ownership.mean())

def seat_utilization(trips):
    veh_1 = int(trips['trip_mode'].isin(['DRIVEALONEFREE','DRIVEALONEPAY']).sum())
    veh_2 = int(trips['trip_mode'].isin(['SHARED2FREE','SHARED2PAY']).sum())
    veh_3 = int(trips['trip_mode'].isin(['SHARED3FREE','SHARED3PAY']).sum())      
    return float((veh_1 + veh_2 + veh_3)/(veh_1 + veh_2/2 + veh_3/3))

def average_traveltime_purpose(trips):
    s = trips.groupby('commute').agg({'travel_time':'mean'})
    return s.to_dict()['travel_time']

def average_traveltime_mode(trips):
    s = trips.groupby('carb_mode').agg({'travel_time':'mean'})
    return s.to_dict()['travel_time']

def average_traveltime_income(trips):
    s = trips.groupby('income_category').agg({'travel_time':'mean'})
    return s.to_dict()['travel_time']

def average_commute_trip_lenght(trips):
    s = trips.groupby('carb_mode').agg({'dist_miles':'mean'})
    return s.to_dict()['dist_miles']


In [ ]:
transit_ridersip(df)

In [ ]:
mode_shares(df)

In [ ]:
average_vehicle_ownership(households)

In [ ]:
seat_utilization(trips)

In [ ]:
average_traveltime_purpose(df)

In [ ]:
average_traveltime_mode(df)

In [ ]:
average_traveltime_income(df)

In [ ]:
average_commute_trip_lenght(df)

In [ ]:
kpi.get_scenario_results('exaple_1', 'ex_6', 'scenario_1')

In [9]:
a = "hola_{}"#.format(scenario = 'hola')

In [10]:
a.format('h')

'hola_h'